# Importing libraries

In [32]:
import numpy as np
from fitizens_libraries.load_and_process_training_data import load_training_data
import os
import pandas as pd
import statistics as stat
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten,Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import F1Score
from sklearn.model_selection import ParameterGrid

# Creating Functions

In [42]:
def requires_peak_minima(exercise: str):
    if exercise == "SQUAT":
        return True
    else:
        return False

In [43]:
def load_data(exercise: str, folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    file_names = [f"{folder_path}/{name}" for name in os.listdir(folder_path)]
    signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    
    data, wk = load_training_data(filelist=file_names,
                             signals= signals,
                              target_exercise=exercise, other_exercises=[], is_peak_minima=requires_peak_minima(exercise))
    return data

In [44]:
def resampling_data(df_input, desired_rows, columns):
    df = df_input[columns]
    if df.shape[0] == desired_rows:
        return df    

    additional_index = pd.date_range(start=df.index.min(),  
                                     end=df.index.max(),
                                     periods=desired_rows)
    resampled_df = df.reindex(additional_index).interpolate()
    
    return resampled_df

In [45]:
def get_training_test_data(exercise_tag: str, folder_path: str, columns_selected: list):
    data = load_data(exercise_tag, folder_path)
    
    exercise = [element["series"] for element in list(filter(lambda info: info["target"] == exercise_tag, data))]
    no_exercise = [element["series"] for element in list(filter(lambda info: info["target"] == "NO_EXERCISE", data))]
    
    exercise_len = len(exercise)
    no_exercise_len = len(no_exercise)
    total_len = exercise_len + no_exercise_len
    exercise_weight = total_len / ( 2 * exercise_len )
    no_exercise_weight = total_len / ( 2* no_exercise_len )
    
    metric = int(stat.median([info.shape[0] for info in exercise]))
    resampled_exercise = [resampling_data(info, metric, columns_selected) for info in exercise]
    resampled_no_exercise = [resampling_data(info, metric, columns_selected) for info in no_exercise]
    
    resampled_exercise_labeled = [1] * len(resampled_exercise)
    resampled_no_exercise_labeled = [0] * len(resampled_no_exercise)
    
    input_data = resampled_exercise + resampled_no_exercise
    input_data = np.stack([df.to_numpy() for df in input_data])
    output_data = np.array(resampled_exercise_labeled + resampled_no_exercise_labeled)
    
    return {"X": input_data, "Y": output_data, "exercise_weight": exercise_weight, "no_exercise_weight": no_exercise_weight, "metric": metric}

In [46]:
def create_model(params, input_shape):
    model = Sequential()
    model.add(Conv1D(filters=params['filters'], kernel_size=params['kernel_size'], 
                     activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(params['dense_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dense(2, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[F1Score()])
    return model

In [47]:
def hyperparameter_tuning(X_train, y_train, X_test, y_test, input_shape, class_weights_dict):
    param_grid = {
        'filters': [32, 64, 128],
        'kernel_size': [3, 5],
        'dense_units': [32, 64, 128],
        'learning_rate': [0.001, 0.0001]
    }

    best_f1 = 0
    best_model = None
    best_params = None

    for params in ParameterGrid(param_grid):
        model = create_model(params, input_shape)
        model.fit(X_train, y_train, epochs=100, batch_size=32, class_weight=class_weights_dict, validation_split=0.2, verbose=0)
        _, f1 = model.evaluate(X_test, y_test, verbose=0)
        f1_exercise = f1[1]
        if f1_exercise > best_f1:
            best_f1 = f1_exercise
            best_model = model
            best_params = params

    return best_model, best_f1, best_params

In [48]:
def predict_deep_learning(exercise_tag: str, folder_path: str, columns_selected: list):
    data = get_training_test_data(exercise_tag, folder_path, columns_selected)
    
    X_train, X_test, y_train, y_test = train_test_split(data["X"], data["Y"], test_size=0.2, random_state=123)
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    class_weights_dict = {0: data["no_exercise_weight"], 1: data["exercise_weight"]}
    input_shape = (data["metric"], len(columns_selected))

    return hyperparameter_tuning(X_train, y_train, X_test, y_test, input_shape, class_weights_dict)

# Testing

In [49]:
features = ["accX", "accY", "accZ", "linAccX", "linAccY", "linAccZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ"]

best_model_generated, best_f1_generated, best_params_generated = predict_deep_learning("SQUAT", "LABELED", features)
best_f1_generated

0.97897893

In [50]:
best_params_generated

{'dense_units': 32, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.0001}

In [52]:
best_model_generated.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 109, 32)           1184      
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 54, 32)            0         
 g1D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 1728)              0         
                                                                 
 dense_12 (Dense)            (None, 32)                55328     
                                                                 
 dense_13 (Dense)            (None, 2)                 66        
                                                                 
Total params: 56578 (221.01 KB)
Trainable params: 56578 (221.01 KB)
Non-trainable params: 0 (0.00 Byte)
________________